Data Preparation Module


In [116]:
import pandas as pd

import threading
import time


try:
    import logging

    import yfinance as yf

    logging.disable(logging.ERROR)

    ENABLE_YFINANCE = True
except ImportError:
    ENABLE_YFINANCE = False

try:
    from tqdm import tqdm

    ENABLE_TQDM = True
except ImportError:
    ENABLE_TQDM = False


In [121]:
def get_historical_data(
    tickers: list[str] | str,
    ticker_type: str = "stock",
    #start: str | None = "01/01/2010",
    #end: str | None = "28/10/2022",
    currency: str | None = "USD",
    source_path: str = "/Users/guilhembarroyer/Desktop/Projects/financial-index-tracker/InputFiles/data.xlsx",
    #show_ticker_seperation: bool = True,
    show_errors: bool = True,
    tqdm_message: str = "Obtaining historical data",
    progress_bar: bool = True,
):
   
    def worker(ticker, historical_data_dict):
        
        historical_data = pd.DataFrame()
       
        
        historical_data = get_historical_data_from_excel(
            ticker=ticker,
            ticker_type=ticker_type,
            currency=currency,
            source_path=source_path,
        )

        if not historical_data==None:
            excel_tickers.append(ticker)
        
        
        #if historical_data.empty or ticker not in excel_tickers:
         #   if ENABLE_YFINANCE:
          #      historical_data = get_historical_data_from_yahoo_finance(
           #         ticker=ticker,
            #        start=start,
             #       end=end,
              #      interval=interval,
               #     return_column=return_column,
                #    risk_free_rate=risk_free_rate,
                 #   divide_ohlc_by=divide_ohlc_by,
                #)

            #if not historical_data.empty:
             #   yf_tickers.append(ticker)

        if historical_data==None:
            no_data.append(ticker)
        if not historical_data==None:
            historical_data_dict[ticker] = historical_data

    if isinstance(tickers, str):
        ticker_list = [tickers]
    elif isinstance(tickers, list):
        ticker_list = tickers
    else:
        raise ValueError(f"Type for the tickers ({type(tickers)}) variable is invalid.")

    ticker_list_iterator = (
        tqdm(ticker_list, desc=tqdm_message)
        if (ENABLE_TQDM & progress_bar)
        else ticker_list
    )

    historical_data_dict: dict[str, pd.DataFrame] = {}
    excel_tickers: list[str] = []
    #yf_tickers: list[str] = []
    no_data: list[str] = []
    threads = []

    for ticker in ticker_list_iterator:
        
        # Introduce a sleep timer to prevent rate limit errors
        time.sleep(0.1)

        thread = threading.Thread(
            target=worker,
            args=(ticker, historical_data_dict),
        )
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()



    #if excel_tickers and yf_tickers and show_ticker_seperation:
     #   print(
      #      f"The following tickers acquired historical data from the xlsx data file: {', '.join(excel_tickers)}"
       # )
       # print(
       #     f"The following tickers acquired historical data from YahooFinance: {', '.join(yf_tickers)}"
       # )

    if no_data and show_errors:
        if not ENABLE_YFINANCE:
            print(
                "Due to a missing optional dependency (yfinance) and the current data file, "
                f"data for the following tickers could not be acquired: {', '.join(no_data)}\n"
                "Enable this functionality by using:\033[1m pip install 'financetoolkit[yfinance]' \033[0m"
            )
        else:
            print(f"No data found for the following tickers: {', '.join(no_data)}")

    print(len(historical_data_dict))
    if len(historical_data_dict) == 0:
        print("No data found in your conditions.")

    reorder_tickers = [ticker for ticker in tickers if ticker in historical_data_dict]
    historical_data_prices_dict = {ticker: historical_data_dict[ticker]["Prices"] for ticker in reorder_tickers}
    historical_data_information_dict = {ticker: historical_data_dict[ticker]["Information"] for ticker in reorder_tickers}  

    if not historical_data_dict:
        raise ValueError("No data found for the given tickers.")

    historical_data_prices = pd.concat(historical_data_prices_dict).unstack(level=0)
    historical_data_prices = historical_data_prices.reindex(reorder_tickers, level=1, axis=1)

    historical_data_informations = pd.concat(historical_data_information_dict).unstack(level=0)
    historical_data_informations = historical_data_informations.reindex(reorder_tickers, level=1, axis=1)

    return historical_data_informations, historical_data_prices, no_data


def get_historical_data_from_excel(ticker, ticker_type, currency, source_path):
    if(source_path):
        in_excel=False
        if ticker_type == "stock":
            years = [2018, 2019, 2020]
            qualitativ_data_result_df=pd.DataFrame(
            columns=[
                "Name",
                "Bics1",
                "Bics2",
                "Bics3",
                "Bics4",
                "Country",
                "PX_TO_BOOK_RATIO_2018",
                "PX_TO_BOOK_RATIO_2019",
                "PX_TO_BOOK_RATIO_2020",
                "PE_RATIO_2018",
                "PE_RATIO_2019",
                "PE_RATIO_2020",
                "CUR_MKT_CAP_2018",
                "CUR_MKT_CAP_2019",
                "CUR_MKT_CAP_2020",
                "EQY_DVD_YLD_IND_2018",
                "EQY_DVD_YLD_IND_2019",
                "EQY_DVD_YLD_IND_2020",
                "DVD_FREQ_2018",
                "DVD_FREQ_2019",
                "DVD_FREQ_2020",
                "EQY_SH_OUT_2018",
                "EQY_SH_OUT_2019",
                "EQY_SH_OUT_2020",
                "PX_LAST_2018",
                "PX_LAST_2019",
                "PX_LAST_2020"
            ],)
        

            stocks_description_df = pd.read_excel(source_path, sheet_name="Members")
            stocks_description_df.columns = ['Index', 'NAME', 'BICS_LEVEL_1_SECTOR_NAME', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME', 'BICS_LEVEL_3_INDUSTRY_NAME', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME', '', 'SXXP Index', 'NAME_bis', 'BICS_LEVEL_1_SECTOR_NAME_bis', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME_bis', 'BICS_LEVEL_3_INDUSTRY_NAME_bis', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME_bis']

            if ticker in stocks_description_df.iloc[:, 0].values:   
                in_excel=True
                index="SPX"
                values = stocks_description_df.loc[stocks_description_df.iloc[:, 0] == ticker, ['NAME', 'BICS_LEVEL_1_SECTOR_NAME', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME', 'BICS_LEVEL_3_INDUSTRY_NAME', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME']].values.flatten()
                qualitativ_data_result_df.loc[0, ["Name", "Bics1", "Bics2", "Bics3", "Bics4"]] = values
        

            elif ticker in stocks_description_df.iloc[:, 7].values:    
                in_excel=True
                index="SXXP"
                values = stocks_description_df.loc[stocks_description_df.iloc[:, 7] == ticker, ['NAME_bis', 'BICS_LEVEL_1_SECTOR_NAME_bis', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME_bis', 'BICS_LEVEL_3_INDUSTRY_NAME_bis', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME_bis']].values.flatten()
                qualitativ_data_result_df.loc[0, ["Name", "Bics1", "Bics2", "Bics3", "Bics4"]] = values
                
                
            else: 
                print(f"Ticker {ticker} not found in the data file")

            if in_excel:
                for year in years:
                    qualitativ_data_df = pd.read_excel(source_path, sheet_name=f"Qualitativ_{year}")

                    target_columns=["Country"]+list(f"{col}_{year}" for col in ['PX_TO_BOOK_RATIO', 'PE_RATIO', 'CUR_MKT_CAP', 'EQY_DVD_YLD_IND', "DVD_FREQ", "EQY_SH_OUT", 'PX_LAST'])
                    
                    if ticker in qualitativ_data_df.iloc[:, 0].values:
                        
                        values = qualitativ_data_df.loc[qualitativ_data_df.iloc[:, 0] == ticker, ['COUNTRY','PX_TO_BOOK_RATIO', 'PE_RATIO', 'CUR_MKT_CAP', 'EQY_DVD_YLD_IND', "DVD_FREQ", "EQY_SH_OUT", 'PX_LAST']].values.flatten()
                        
                        qualitativ_data_result_df.loc[0, target_columns] = values


                px_data_df=pd.read_excel(source_path, sheet_name=f"{index}_PX_LAST")
                
                px_data_results_df=pd.DataFrame(columns=["Dates", "Prices"])
                px_data_results_df["Dates"]=px_data_df.iloc[:,1]
                px_data_results_df["Prices"]=px_data_df[ticker]

                data_results={"Information": qualitativ_data_result_df, "Prices": px_data_results_df}
                
                return data_results
            
        elif ticker_type == "index":
            
            historical_data=pd.DataFrame(
            columns=[
                "Dates",
                "Index"
            ],)
            index_description_df = pd.read_excel(source_path, sheet_name="Index")
            historical_data["Dates"] = index_description_df["PX_LAST"]
            if ticker == "SPX":   
                historical_data["Index"] = index_description_df["SPX Index"]
                in_excel=True
            elif ticker == "SXXP":    
                historical_data["Index"] = index_description_df["SXXP Index"]
                in_excel=True
            else: 
                print(f"Ticker {ticker} not found in the data file")
            
            if in_excel:
                historical_data=historical_data.dropna()
                historical_data=historical_data.iloc[::-1].reset_index(drop=True)
                return {"Prices": historical_data}

        return None
        

get_historical_data(tickers=["AAPL UW", "1COV GY", "OK"])


Obtaining historical data: 100%|██████████| 3/3 [00:00<00:00,  6.13it/s]


Ticker OK not found in the data file
Due to a missing optional dependency (yfinance) and the current data file, data for the following tickers could not be acquired: OK
Enable this functionality by using: pip install 'financetoolkit[yfinance]' 
2


(        Name                    Bics1             \
      AAPL UW      1COV GY     AAPL UW    1COV GY   
 0  APPLE INC  COVESTRO AG  Technology  Materials   
 
                             Bics2                           Bics3             \
                           AAPL UW    1COV GY              AAPL UW    1COV GY   
 0  Tech Hardware & Semiconductors  Materials  Technology Hardware  Chemicals   
 
                       Bics4                                 ...  \
                     AAPL UW                        1COV GY  ...   
 0  Communications Equipment  Basic & Diversified Chemicals  ...   
 
   EQY_SH_OUT_2019         EQY_SH_OUT_2020         PX_LAST_2018          \
           AAPL UW 1COV GY         AAPL UW 1COV GY      AAPL UW 1COV GY   
 0        17773.06   183.0       17001.802   193.2       39.058   43.18   
 
   PX_LAST_2019         PX_LAST_2020          
        AAPL UW 1COV GY      AAPL UW 1COV GY  
 0        72.88   41.45       133.72   50.48  
 
 [1 rows x 54 colu

In [114]:
def get_historical_data_from_excel(ticker, ticker_type, currency, source_path):
    if(source_path):
        in_excel=False
        if ticker_type == "stock":
            years = [2018, 2019, 2020]
            qualitativ_data_result_df=pd.DataFrame(
            columns=[
                "Name",
                "Bics1",
                "Bics2",
                "Bics3",
                "Bics4",
                "Country",
                "PX_TO_BOOK_RATIO_2018",
                "PX_TO_BOOK_RATIO_2019",
                "PX_TO_BOOK_RATIO_2020",
                "PE_RATIO_2018",
                "PE_RATIO_2019",
                "PE_RATIO_2020",
                "CUR_MKT_CAP_2018",
                "CUR_MKT_CAP_2019",
                "CUR_MKT_CAP_2020",
                "EQY_DVD_YLD_IND_2018",
                "EQY_DVD_YLD_IND_2019",
                "EQY_DVD_YLD_IND_2020",
                "DVD_FREQ_2018",
                "DVD_FREQ_2019",
                "DVD_FREQ_2020",
                "EQY_SH_OUT_2018",
                "EQY_SH_OUT_2019",
                "EQY_SH_OUT_2020",
                "PX_LAST_2018",
                "PX_LAST_2019",
                "PX_LAST_2020"
            ],)
        

            stocks_description_df = pd.read_excel(source_path, sheet_name="Members")
            stocks_description_df.columns = ['Index', 'NAME', 'BICS_LEVEL_1_SECTOR_NAME', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME', 'BICS_LEVEL_3_INDUSTRY_NAME', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME', '', 'SXXP Index', 'NAME_bis', 'BICS_LEVEL_1_SECTOR_NAME_bis', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME_bis', 'BICS_LEVEL_3_INDUSTRY_NAME_bis', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME_bis']

            if ticker in stocks_description_df.iloc[:, 0].values:   
                in_excel=True
                index="SPX"
                values = stocks_description_df.loc[stocks_description_df.iloc[:, 0] == ticker, ['NAME', 'BICS_LEVEL_1_SECTOR_NAME', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME', 'BICS_LEVEL_3_INDUSTRY_NAME', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME']].values.flatten()
                qualitativ_data_result_df.loc[0, ["Name", "Bics1", "Bics2", "Bics3", "Bics4"]] = values
        

            elif ticker in stocks_description_df.iloc[:, 7].values:    
                in_excel=True
                index="SXXP"
                values = stocks_description_df.loc[stocks_description_df.iloc[:, 7] == ticker, ['NAME_bis', 'BICS_LEVEL_1_SECTOR_NAME_bis', 'BICS_LEVEL_2_INDUSTRY_GROUP_NAME_bis', 'BICS_LEVEL_3_INDUSTRY_NAME_bis', 'BICS_LEVEL_4_SUB_INDUSTRY_NAME_bis']].values.flatten()
                qualitativ_data_result_df.loc[0, ["Name", "Bics1", "Bics2", "Bics3", "Bics4"]] = values
                
                
            else: 
                print(f"Ticker {ticker} not found in the data file")

            if in_excel:
                for year in years:
                    qualitativ_data_df = pd.read_excel(source_path, sheet_name=f"Qualitativ_{year}")

                    target_columns=["Country"]+list(f"{col}_{year}" for col in ['PX_TO_BOOK_RATIO', 'PE_RATIO', 'CUR_MKT_CAP', 'EQY_DVD_YLD_IND', "DVD_FREQ", "EQY_SH_OUT", 'PX_LAST'])
                    
                    if ticker in qualitativ_data_df.iloc[:, 0].values:
                        
                        values = qualitativ_data_df.loc[qualitativ_data_df.iloc[:, 0] == ticker, ['COUNTRY','PX_TO_BOOK_RATIO', 'PE_RATIO', 'CUR_MKT_CAP', 'EQY_DVD_YLD_IND', "DVD_FREQ", "EQY_SH_OUT", 'PX_LAST']].values.flatten()
                        
                        qualitativ_data_result_df.loc[0, target_columns] = values


                px_data_df=pd.read_excel(source_path, sheet_name=f"{index}_PX_LAST")
                
                px_data_results_df=pd.DataFrame(columns=["Dates", "Prices"])
                px_data_results_df["Dates"]=px_data_df.iloc[:,1]
                px_data_results_df["Prices"]=px_data_df[ticker]

                data_results={"Information": qualitativ_data_result_df, "Prices": px_data_results_df}
                
                return data_results
            
        elif ticker_type == "index":
            
            historical_data=pd.DataFrame(
            columns=[
                "Dates",
                "Index"
            ],)
            index_description_df = pd.read_excel(source_path, sheet_name="Index")
            historical_data["Dates"] = index_description_df["PX_LAST"]
            if ticker == "SPX":   
                historical_data["Index"] = index_description_df["SPX Index"]
            elif ticker == "SXXP":    
                historical_data["Index"] = index_description_df["SXXP Index"]
            else: 
                print(f"Ticker {ticker} not found in the data file")
            
            historical_data=historical_data.dropna()
            historical_data=historical_data.iloc[::-1].reset_index(drop=True)


                                         
    



get_historical_data_from_excel("AAPL UW","stock", "currency", "/Users/guilhembarroyer/Desktop/Projects/financial-index-tracker/InputFiles/data.xlsx")



{'Information':         Name       Bics1                           Bics2                Bics3  \
 0  APPLE INC  Technology  Tech Hardware & Semiconductors  Technology Hardware   
 
                       Bics4 Country PX_TO_BOOK_RATIO_2018  \
 0  Communications Equipment      US              6.436164   
 
   PX_TO_BOOK_RATIO_2019 PX_TO_BOOK_RATIO_2020 PE_RATIO_2018  ...  \
 0              7.121414             37.682547     12.256562  ...   
 
   EQY_DVD_YLD_IND_2020 DVD_FREQ_2018 DVD_FREQ_2019 DVD_FREQ_2020  \
 0               0.6132       Quarter       Quarter       Quarter   
 
   EQY_SH_OUT_2018 EQY_SH_OUT_2019 EQY_SH_OUT_2020 PX_LAST_2018 PX_LAST_2019  \
 0       18981.592        17773.06       17001.802       39.058        72.88   
 
   PX_LAST_2020  
 0       133.72  
 
 [1 rows x 27 columns],
 'Prices':           Dates   Prices
 0    2010-01-04    7.649
 1    2010-01-05    7.656
 2    2010-01-06    7.535
 3    2010-01-07    7.521
 4    2010-01-08    7.571
 ...         ...      .